In [3]:
#https://data.torontopolice.on.ca/datasets/TorontoPS::ksi/about

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

mvc = pd.read_csv('KSI_MVC.csv',low_memory=False)

In [4]:
mvc.head()

,X,Y,INDEX_,ACCNUM,YEAR,DATE,TIME,STREET1,STREET2,OFFSET,...,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,DIVISION,ObjectId
0,-8844610.729,5412413.888,3387730,892658.0,2006,2006/03/11 05:00:00+00,852,BLOOR ST W,DUNDAS ST W,NaN,...,Yes,NaN,NaN,NaN,88,High Park North,88,High Park North (88),D11,1
1,-8844610.729,5412413.888,3387731,892658.0,2006,2006/03/11 05:00:00+00,852,BLOOR ST W,DUNDAS ST W,NaN,...,Yes,NaN,NaN,NaN,88,High Park North,88,High Park North (88),D11,2
2,-8816479.848,5434843.389,3388101,892810.0,2006,2006/03/11 05:00:00+00,915,MORNINGSIDE AVE,SHEPPARD AVE E,NaN,...,Yes,Yes,NaN,NaN,146,Malvern East,132,Malvern (132),D42,3
3,-8829728.092,5419070.899,3389067,893184.0,2006,2006/01/01 05:00:00+00,236,WOODBINE AVE,O CONNOR DR,NaN,...,Yes,NaN,Yes,NaN,60,Woodbine-Lumsden,60,Woodbine-Lumsden (60),D55,4
4,-8816479.848,5434843.389,3388102,892810.0,2006,2006/03/11 05:00:00+00,915,MORNINGSIDE AVE,SHEPPARD AVE E,NaN,...,Yes,Yes,NaN,NaN,146,Malvern East,132,Malvern (132),D42,5


In [5]:
mvc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18194 entries, 0 to 18193
Data columns (total 57 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   X                  18194 non-null  float64
 1   Y                  18194 non-null  float64
 2   INDEX_             18194 non-null  int64  
 3   ACCNUM             13264 non-null  float64
 4   YEAR               18194 non-null  int64  
 5   DATE               18194 non-null  object 
 6   TIME               18194 non-null  int64  
 7   STREET1            18194 non-null  object 
 8   STREET2            16510 non-null  object 
 9   OFFSET             3402 non-null   object 
 10  ROAD_CLASS         17818 non-null  object 
 11  DISTRICT           18089 non-null  object 
 12  WARDNUM            17332 non-null  float64
 13  LATITUDE           18194 non-null  float64
 14  LONGITUDE          18194 non-null  float64
 15  LOCCOORD           18099 non-null  object 
 16  ACCLOC             127

In [6]:
mvc.describe(include='all')

,X,Y,INDEX_,ACCNUM,YEAR,DATE,TIME,STREET1,STREET2,OFFSET,...,AG_DRIV,REDLIGHT,ALCOHOL,DISABILITY,HOOD_158,NEIGHBOURHOOD_158,HOOD_140,NEIGHBOURHOOD_140,DIVISION,ObjectId
count,1.819400e+04,1.819400e+04,1.819400e+04,1.326400e+04,18194.000000,18194,18194.000000,18194,16510,3402,...,9460,1520,788,486,18194,18194,18194,18194,18194,18194.000000
unique,NaN,NaN,NaN,NaN,NaN,3925,NaN,1856,2704,477,...,1,1,1,1,159,159,141,141,17,NaN
top,NaN,NaN,NaN,NaN,NaN,2014/08/17 04:00:00+00,NaN,YONGE ST,BATHURST ST,10 m North of,...,Yes,Yes,Yes,Yes,1,West Humber-Clairville,77,Waterfront Communities-The Island (77),D42,NaN
freq,NaN,NaN,NaN,NaN,NaN,35,NaN,378,152,58,...,9460,1520,788,486,579,579,703,703,1760,NaN
mean,-8.838345e+06,5.420748e+06,3.818870e+07,4.248444e+08,2012.934869,NaN,1362.615917,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9097.500000
std,1.162533e+04,8.682160e+03,3.726463e+07,1.065503e+09,4.754258,NaN,630.816048,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5252.299734
min,-8.865305e+06,5.402162e+06,3.363207e+06,2.530100e+04,2006.000000,NaN,0.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000
25%,-8.846591e+06,5.413242e+06,5.391370e+06,1.021229e+06,2009.000000,NaN,920.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4549.250000
50%,-8.838448e+06,5.419556e+06,7.644612e+06,1.197308e+06,2012.000000,NaN,1450.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9097.500000
75%,-8.829671e+06,5.427813e+06,8.078261e+07,1.365020e+06,2017.000000,NaN,1850.000000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13645.750000


In [7]:
mvc.duplicated().value_counts()

False    18194
dtype: int64

In [8]:
mvc.columns.to_list()

['X',
 'Y',
 'INDEX_',
 'ACCNUM',
 'YEAR',
 'DATE',
 'TIME',
 'STREET1',
 'STREET2',
 'OFFSET',
 'ROAD_CLASS',
 'DISTRICT',
 'WARDNUM',
 'LATITUDE',
 'LONGITUDE',
 'LOCCOORD',
 'ACCLOC',
 'TRAFFCTL',
 'VISIBILITY',
 'LIGHT',
 'RDSFCOND',
 'ACCLASS',
 'IMPACTYPE',
 'INVTYPE',
 'INVAGE',
 'INJURY',
 'FATAL_NO',
 'INITDIR',
 'VEHTYPE',
 'MANOEUVER',
 'DRIVACT',
 'DRIVCOND',
 'PEDTYPE',
 'PEDACT',
 'PEDCOND',
 'CYCLISTYPE',
 'CYCACT',
 'CYCCOND',
 'PEDESTRIAN',
 'CYCLIST',
 'AUTOMOBILE',
 'MOTORCYCLE',
 'TRUCK',
 'TRSN_CITY_VEH',
 'EMERG_VEH',
 'PASSENGER',
 'SPEEDING',
 'AG_DRIV',
 'REDLIGHT',
 'ALCOHOL',
 'DISABILITY',
 'HOOD_158',
 'NEIGHBOURHOOD_158',
 'HOOD_140',
 'NEIGHBOURHOOD_140',
 'DIVISION',
 'ObjectId']